In [ ]:
import warnings
import pandas as pd
import arviz as az

import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)
import calendar
from libs.model import HGPforecaster
from libs.metrics import calculate_metrics, metrics_to_table
from libs.pre_processing import generate_groups_data_flat, generate_groups_data_matrix
from libs.visual_analysis import visualize_fit, visualize_predict, visualize_prior, model_graph, traceplot, plot_gps_components, plot_elbo
az.style.use('arviz-darkgrid')
warnings.filterwarnings('ignore')

# Minibatch - Outside Piecewise Linear w/ Partial pooling

In [ ]:
data = pd.read_csv('../data/TourismData_v3.csv')
data['Year'] = data['Year'].fillna(method='ffill')

d = dict((v,k) for k,v in enumerate(calendar.month_name))
data.Month = data.Month.map(d)
data = data.assign(t=pd.to_datetime(data[['Year', 'Month']].assign(day=1))).set_index('t')
data = data.drop(['Year', 'Month'], axis=1)
data = data.round()

groups_input = {
    'state': [0,1],
    'zone': [0,2],
    'region': [0,3],
    'purpose': [3,6]
}

In [ ]:
groups = generate_groups_data_flat(y = data, 
                               groups_input = groups_input, 
                               seasonality=12, 
                               h=24)

In [ ]:
m = HGPforecaster(groups_data=groups,
                  n_iterations=100000,
                  changepoints = 4,
                  partial_pool=True,
                  minibatch=[100,20])

In [ ]:
m.fit_vi()

In [ ]:
m.predict()

In [ ]:
plot_elbo(m.trace_vi, 50000)

In [ ]:
plot_gps_components(series=31, groups=m.g, trace=m.trace_vi_samples)

In [ ]:
visualize_predict(groups, m.pred_samples_predict, 8)

In [ ]:
results = calculate_metrics(m.pred_samples_predict, groups)
metrics_to_table(results)